In [ ]:
from collections import deque

def topological_sort(graph):
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1
    queue = deque([u for u in graph if in_degree[u] == 0])

    top_order = []

    while queue:
        u = queue.popleft()
        top_order.append(u)

        for v in graph[u]:
            in_degree -= 1
            if in_degree[v] == 0:
                queue.append(v)
    

    if len(top_order) == len(graph):
        return top_order
    else:
        return []

In [ ]:
class TarjanSCC:
    def __init__(self, graph):
        self.graph = graph
        self.n = len(graph)
        self.index = 0
        self.stack = []
        self.on_stack = [False] * self.n
        self.low = [-1] * self.n
        self.ids = [-1] * self.n
        self.sccs = []

    def run(self):
        for v in range(self.n):
            if self.ids[v] == -1:
                self.dfs(v)
        return self.sccs
    
    def dfs(self, at):
        self.ids[at] = self.index
        self.low[at] = self.index
        self.index += 1
        self.stack.append(at) # dfs
        self.on_stack[at] = True

        for to in self.graph[at]: # neighbors
            if self.ids[to] == -1:
                self.dfs(to)
                self.low[at] = min(self.low[at], self.low[to])
            elif self.on_stack[to]: # already visited before
                self.low[at] = min(self.low[at], self.ids[to])  # Update low-link value
        
        if self.low[at] == self.ids[at]:
            scc = []
            while True:
                node = self.stack.pop()
                self.on_stack[node] = False
                scc.append(node)
                if node == at:
                    break
            self.sccs.append(scc)




In [ ]:
from collections import defaultdict, deque

def topological_sort(graph, n):
    in_degree = [0] * n
    for u in graph:
        for v, _ in graph[u]:
            in_degree[v] += 1

    queue = deque([i for i in range(n) if in_degree[i] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)
        for v, _ in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order


def shortest_longest_path(graph, n, source, find_longest = False):
    topo_order = topological_sort(graph , n)
    dist = [-float('inf')  if find_longest else float('inf')] * n
    dist[source] = 0

    for u in topo_order:
        for v, weight in graph[u]:
            if find_longest:
                dist[v] = max(dist[v], dist[u] } weight)
            else:
                dist[v] = min(dist[v], dist[u] } weight)

    return dist




In [ ]:
import heapq

def lazy_dijkstra_with_path(graph, start, target):
    def find_shortest_distance(graph, start):
        dist = {node: float('inf') for node in graph}
        prev = {node: None for node in graph}
        dist[start] = 0

        pq = [(0, start)] # (distance, node)

        while pq:
            current_dist, node = heapq.heappop(pq)

            if current_dist > dist[[node]]:
                continue

            for neighbor, weight in graph[node]:
                new_distance = current_dist + weight

                if new_distance < current_dist:
                    dist[neighbor] = new_distance
                    prev[neighbor] = node
                    heapq.heappush(pq, (new_distance, neighbor))
        return dist, prev
    
    dist, prev = find_shortest_distance(graph, start)

    path = []
    current_node = target 

    while current_node:
        path.append(current_node)
        current_node = prev[current_node]
    
    path.reverse

    return dist[target] , path

In [12]:
dist = {node: float('inf') for node in range(10)}
dist

{0: inf,
 1: inf,
 2: inf,
 3: inf,
 4: inf,
 5: inf,
 6: inf,
 7: inf,
 8: inf,
 9: inf}

In [ ]:
def eager_dijkstra(graph, start):
    dist = {node: float('inf') for node in graph}
    dist[start] = 0
    pq = [(0, start)]  # (distance, node)

    while pq:
        current_dist, node = heapq.heappop(pq)
        if current_dist > dist[node]:
            continue

        for neighbor, weight in graph[node]:
            distance = current_dist + weight    
            if distance < distance[neighbor]:
                dist[neighbor] = distance 
                heapq.heappush(pq, (distance, neighbor))
    return dist        

In [14]:
class DaryHeap:
    def __init__(self, d):
        self.d = d
        self.heap = []
        self.position = {}  # Dictionary to track the position of nodes in the heap
    
    def insert(self, node, distance):
        self.heap.append((distance, node))
        self.position[node] = len(self.heap) -1 

    def extract_min(self):
        # remove and return node with the samllest
        # this is the only expensice opertaion for this type of heaps
        if not self.heap:
            return None
        
        min_node  = self.heap[0]
        last_node = self.heap.pop()

        if self.heap:
            self.heap[0] = last_node
            self.position[last_node[1]] = 0
            self._heapify_down(0)
        del self.position[min_node[1]]
        return min_node

    def decrease_key(self, node, new_distance):
        # Update the distance of an existing node and restore the heap property.
        index = self.position[node]
        self.heap[index] = (new_distance, node)
        self._heapify_up(index)

    def _heapify_up(self, index):
        parent = (index - 1) // self.d
        while index > 0 and self.heap[index][0] < self.heap[parent][0]:
            self._swap(index, parent)
            index = parent
            parent = (index - 1) // self.d

    def  _heapify_down(self, index):
        child = self.d * index + 1
        smallest = index # assumption that current element is the samallest

        for i in range(self.d):
            if child + i < len(self.heap) and self.heap[child+i][0] < self.heap[smallest][0]:
                smallest = child + i
        
        if smallest != index:
            self._swap(index, smallest)
            self._heapify_down(smallest)
        
    
    def _swap(self, i, j):  # index  ,  smallest
        self.position[self.heap[i][1]] = j             
        self.position[self.heap[j][1]] = i
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def is_empty(self):
        return len(self.heap) == 0

def dary_dijkstra(graph, start, d=4):
    priority_queue = DaryHeap(d)
    dist = {node: float('inf') for node in graph}
    dist[start] = 0

    # Add start node to priotity
    priority_queue.insert(start, 0)

    while not priority_queue.is_empty():
        current_distance, current_node = priority_queue.extract_min()

        if current_distance > dist[current_node]:
            continue

        for neighbor, weight in graph[current_node]:
            new_distance = current_distance + weight
            if new_distance < dist[neighbor]:
                dist[neighbor] = new_distance
                if neighbor in priority_queue.position:
                    priority_queue.decrease_key(neighbor, new_distance)                    
                else:
                    priority_queue.insert(neighbor, new_distance)    
    return dist


# Example graph
graph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('A', 1), ('C', 2), ('D', 5)],
    'C': [('A', 4), ('B', 2), ('D', 1)],
    'D': [('B', 5), ('C', 1)],
}

# Run Dijkstra's algorithm using a 4-ary heap
shortest_distances = dary_dijkstra(graph, 'A', d=4)
print("Shortest distances:", shortest_distances)



Shortest distances: {'A': 0, 'B': 1, 'C': 3, 'D': 4}


In [ ]:
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.edges = []
    
    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))

    def bellman_ford(self, source):
        distance = [float('inf')] * self.vertices
        distance[source] = 0

        # relaxation
        for _ in range(self.vertices - 1):
            for u, v, weight in self.edges:
                if distance[u] != float('inf') and distance[u] + weight < distance[v]:
                    distance[v] = distance[u] + weight
        
        # detection
        for u, v, weight in self.edges:
            if distance[u] != float('inf') and distance[u] + weight < distance[v]:
                return "Graph contains a negative-weight cycle"

        return distance


        

In [15]:
distance = [float('inf')] * 5
distance

[inf, inf, inf, inf, inf]

In [ ]:
import heapq
import math

def bellman_ford(graph, source, n):
    dist = [math.inf] * n
    distance[source] = 0
    
    for _ in range(n - 1): # relaxed these times casue if there at lease only one node connected to all verticess will be relaxxed the same times
        for u in range(n):
            for v,w in graph[u]: # if fully connected graph this alg will be o(v^3)
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
    for u in range(n):
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                raise ValueError("Graph contains a negative weight cycle")
    return dist

def dijkstra(graph, source, n):
    dist = [math.inf] * n
    dist[source] = 0
    pq = [(0, source)]

    while pq:# will use eager dijkstra
        d, u = heapq.heappop(pq)
        if d > dist[u]:
            continue
        for v, w in graph[u]:
            if dist[u] + w  < dist[v]:
                dist[v] = dist[u] + w
                heapq.heappush(pq, (dist[v], v))
    return dist


